In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import json
import os

In [2]:
def load_properties():
        with open( 'C:/Users/Mohd.Humza/Desktop/Notebook/gcp/Srinisalesapp2/properties/properties.json' ) as fin:
                SETTING = json.load( fin )
        return SETTING

In [3]:
def env():
    SETTING = load_properties()
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SETTING['KEY_FILE']
    credentials = service_account.Credentials.from_service_account_file(
                                        'C:/Users/Mohd.Humza/Desktop/Notebook/gcp/Srinisalesapp2/credentials/orderanalyticsapp1-b6263ba87422.json')
    bqclient = bigquery.Client(credentials=credentials,project=SETTING['PROJECT'],)
    return credentials,bqclient

In [ ]:
def transformation(dataframe):
    ss = dataframe
    ss = ss.fillna(0)
    filter_values = ['S','X','R119']
    ss = ss[ss['SJCd'].str.contains('|'.join(filter_values), na=False)]
    ss['FinancialDayDt'] = ss['FinancialDayDt'].replace('-','')
    return ss

In [10]:
def read_from_table():
    credentials,bqclient = env()
    table_ss = bigquery.TableReference.from_string("orderanalyticsapp1.salessummarydata.salessummary_stage")
    rows_ss = bqclient.list_rows(table_ss,selected_fields=[
        bigquery.SchemaField("SlNo", "STRING"),
        bigquery.SchemaField("RevenueLocNo", "STRING"),
        bigquery.SchemaField("LocNo", "STRING"),
        bigquery.SchemaField("TrxSetDt", "STRING"),
        bigquery.SchemaField("FinancialDayDt", "STRING"),
        bigquery.SchemaField("SourceSystem", "STRING"),
        bigquery.SchemaField("SJCd", "STRING"),
        bigquery.SchemaField("Amount", "STRING"),
        bigquery.SchemaField("ClickSold", "STRING"),
        bigquery.SchemaField("BusinessUnitCd", "STRING"),
        bigquery.SchemaField("QuantitySold", "STRING"),],)
    dataframe_ss = rows_ss.to_dataframe()
    table_loc = bigquery.TableReference.from_string("orderanalyticsapp1.salessummarydata.dwd_location")
    rows_loc = bqclient.list_rows(table_loc,selected_fields=[
        bigquery.SchemaField("LocId", "INTEGER"),
        bigquery.SchemaField("LocNo", "STRING"),
        bigquery.SchemaField("CurrentFl", "STRING"),
        bigquery.SchemaField("LocDesc", "STRING"),
        bigquery.SchemaField("BranchYN", "STRING"),
        bigquery.SchemaField("StateCd", "STRING"),
        bigquery.SchemaField("CountryCd", "STRING"),
        bigquery.SchemaField("OpenDt", "STRING"),
        bigquery.SchemaField("CloseDt", "STRING"),
        bigquery.SchemaField("Level1Cd", "STRING"),
        bigquery.SchemaField("Level1Desc", "STRING"),
        bigquery.SchemaField("Level2Cd", "STRING"),
        bigquery.SchemaField("Level2Desc", "STRING"),
        bigquery.SchemaField("Level3Cd", "STRING"),
        bigquery.SchemaField("Level3Desc", "STRING"),
        bigquery.SchemaField("Level4Cd", "STRING"),
        bigquery.SchemaField("Level4Desc", "STRING"),
        bigquery.SchemaField("Level5Cd", "STRING"),
        bigquery.SchemaField("Level5Desc", "STRING"),
        bigquery.SchemaField("Level6Cd", "STRING"),
        bigquery.SchemaField("Level6Desc", "STRING"),
        bigquery.SchemaField("CountryDesc", "STRING"),],)
    dataframe_loc = rows_loc.to_dataframe()
    return dataframe_ss,dataframe_loc
    
    #dataframe = transformation(dataframe_ss)
    #pandas_gbq.context.credentials = credentials
    #pandas_gbq.to_gbq(dataframe,destination_table='salessummarydata.dwf_daily_branch_sales',project_id='orderanalyticsapp1',credentials=credentials,
                     #if_exists='replace')

In [11]:
if __name__ == "__main__":
    dataframe_ss,dataframe_loc = read_from_table()

In [12]:
dataframe_ss.head()

,SlNo,RevenueLocNo,LocNo,TrxSetDt,FinancialDayDt,SourceSystem,SJCd,Amount,ClickSold,BusinessUnitCd,QuantitySold
0,None,None,None,None,None,None,None,None,None,None,None
1,4,0,4,2019-10-22,2019-10-22,FPKG,S130,7.38,0,10043,0
2,5,0,5,2019-10-23,2019-10-23,FPKG,S112,-11.58,0,10043,0
3,7,0,7,2019-10-25,2019-10-25,FPKG,R112,19.37,0,10043,0
4,6,0,6,2019-10-24,2019-10-24,FPKG,R119,71.35,1,10043,9


In [13]:
dataframe_loc.head()

,LocId,LocNo,CurrentFl,LocDesc,BranchYN,StateCd,CountryCd,OpenDt,CloseDt,Level1Cd,...,Level2Desc,Level3Cd,Level3Desc,Level4Cd,Level4Desc,Level5Cd,Level5Desc,Level6Cd,Level6Desc,CountryDesc
0,497,3629,Y,Oak Lawn IL Cicero,Y,IL,,05-01-1995 00:00:00,,SA0049,...,Dist 49 Chicago South,049,Reg 49 Ops-Midwest,20,East Area,01,Field Operations,10043,FedEx Office,USA
1,3030,1726,Y,Cincinnati OH Colerain,Y,OH,,01-14-2008 00:00:00,,SA0069,...,Dist 69 Cincinnati,048,Reg 48 Ops-Ohio Valley,20,East Area,01,Field Operations,10043,FedEx Office,USA
2,3375,2146,Y,Edgewater MD SR 2 at Maryland,Y,MD,,,01-31-2010 00:00:00,SA0065,...,Dist 65 Baltimore,045,Reg 45 Ops-Mid Atlantic,20,East Area,01,Field Operations,10043,FedEx Office,USA
3,4899,1371,Y,Miami FL Ritz-Carlton Key Bisc,Y,,,,,SA0056,...,Dist 56 Miami South,047,Reg 47 Ops-Southeast,20,East Area,01,Field Operations,10043,FedEx Office,USA
4,3460,2238,Y,Winter Springs FL RedBugLakeRd,Y,FL,,02-25-2008 00:00:00,,SA0058,...,Dist 58 Orlando North,047,Reg 47 Ops-Southeast,20,East Area,01,Field Operations,10043,FedEx Office,USA


In [16]:
'''SlNo = []
RevenueLocNo = []
LocNo = []
TrxSetDt = []
FinancialDayDt = []
SourceSystem = []
SJCd = []
Amount = []
ClickSold = []
BusinessUnitCd = []
QuantitySold = []'''
import pandas as pd
import numpy as np
newData = pd.DataFrame()
newData['LocNo'] = np.where(dataframe_ss['RevenueLocNo']==dataframe_loc['LocNo'] and dataframe_loc['Level6Cd']==dataframe_ss['RevenueLocNo'])
newData['TrxSetDt'] = dataframe_ss['FinancialDayDt'].replace('-','')



ValueError: Can only compare identically-labeled Series objects

In [ ]:
def CustPivot(dataP):
    threshold = 0.01
    pnums = []
    dates = []
    numAnomalies = []
    numNormal = []
    totalForDay = []
    fractAnomalous = []
    for pnum in dataP.payr_cust_nbr.unique():
        dataTemp = dataP[dataP.payr_cust_nbr == pnum]
        dataTemp = dataTemp.sort_values(['shp_dt'], ascending = True)
        print(pnum, len(dataTemp.DiscountPercent.unique()))
        for date in dataTemp.shp_dt.unique():
            dataTempDate = dataTemp[dataTemp.shp_dt == date]
            numLessThanThreshold = len(dataTempDate[dataTempDate.DiscountPercent <= threshold])
            numMoreThanThreshold = len(dataTempDate[dataTempDate.DiscountPercent > threshold])
            pnums.append(pnum)
            dates.append(date)
            numAnomalies.append(numLessThanThreshold)
            numNormal.append(numMoreThanThreshold)
            totalForDay.append(numLessThanThreshold + numMoreThanThreshold)
            try:
                fractAnomalous.append(float(numLessThanThreshold)/(numLessThanThreshold + numMoreThanThreshold))
            except ZeroDivisionError:
                print "A ZeroDivisionError Occured"
                fractAnomalous.append(int(0))
                
    dataResults = pd.DataFrame()
    dataResults['payr_cust_nbr'] = pnums
    dataResults['date'] = dates 
    dataResults['numAnomalies'] = numAnomalies
    dataResults['numNormal'] = numNormal
    dataResults['totalForDay'] = totalForDay
    dataResults['fractAnomalous'] = fractAnomalous
    return dataResults
